In [1]:
import pyfair

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import numpy as np
import os

In [4]:
model1 = pyfair.FairModel(name="HA", n_simulations=10_000)
model1.input_data('Loss Event Frequency', low=.01, mode=.2, high=.5)
model1.input_data('Probable Loss Magnitude', low=1, mode=1_000_000, high=10_000_000)
model1.calculate_all()

model2 = pyfair.FairModel(name="NOT HA", n_simulations=10_000)
model2.input_data('Loss Event Frequency', low=.01, mode=.2, high=.5)
model2.input_data('Probable Loss Magnitude', low=1, mode=1_000_000, high=10_000_000)
model2.calculate_all()


meta1 = pyfair.FairMetaModel('Metamodel', [model1, model2])
out_json = pyfair.FairMetaModel('Metamodel2', [model1, meta1])

print(out_json.to_json())

{
    "HA": {
        "Creation Datetime": "2019-05-12 22:27:34.770695",
        "Loss Event Frequency": {
            "low": 0.01,
            "mode": 0.2,
            "high": 0.5
        },
        "Probable Loss Magnitude": {
            "low": 1,
            "mode": 1000000,
            "high": 10000000
        },
        "name": "HA",
        "n_simulations": 10000,
        "random_seed": 34,
        "model_uuid": "0c918c98-752f-11e9-9dde-f06e0bbd6cbe",
        "type": "<class 'pyfair.model.model.FairModel'>"
    },
    "NOT HA": {
        "Creation Datetime": "2019-05-12 22:27:34.870259",
        "Loss Event Frequency": {
            "low": 0.01,
            "mode": 0.2,
            "high": 0.5
        },
        "Probable Loss Magnitude": {
            "low": 1,
            "mode": 1000000,
            "high": 10000000
        },
        "name": "NOT HA",
        "n_simulations": 10000,
        "random_seed": 34,
        "model_uuid": "0c9b36c8-752f-11e9-b396-f06e0bbd6cbe",
    

In [4]:
str(type(meta1))

"<class 'pyfair.model.meta_model.FairMetaModel'>"

In [5]:
import pandas as pd

import os


metadata = pd.Series(
    {
        'Author': os.environ['USERNAME'],
        'Created': str(pd.datetime.now()).partition('.')[0],
        'PyFair Version': pyfair.VERSION
    }
).to_frame().to_html(header=None, justify='left', classes='mystyle')

STYLE = r'''

#metadata_table {
    display: flex;
}

.mystyle {
    font-size: 11pt; 
    font-family: Arial;
    border-collapse: collapse; 
    border: 1px solid silver;

}

.mystyle td, th {
    padding: 2px;
}

.mystyle tr:nth-child(even) {
    background: #E0E0E0;
}

.mystyle tr:hover {
    background: silver;
    cursor: pointer;
}

.mystyle th {
    text-align: left
}

#band {
    background-color: navy;
    display: flex;
    flex-direction: row;
    width: 100%;
}

#metadata_and_canvas {
    display: flex;
    flex-direction: row;
    width: 100%;
    padding: 10px 0px 10px 0px;
}

#title {
    color: white;
    font-family: Arial;
    text-align: center;
    vertical-align: middle;
}

#python_img_container {
    align-self: center;
    text-align: center;
}

#python_img {
    width: 40%;
}

#tree_diagram {
    border:1px solid silver;
}

#main {
    display: flex;
    flex-direction: column;
}

'''


HTML_REPORT = f'''

<head>

<style>{STYLE}</style>

</head>

<body>
    <div id="main">
        <div id="band">
            <div id="python_img_container">
                <img id="python_img" src="C:/Users/theon/Desktop/white_python_logo.svg">
            </div>
            <div style="flex-grow: 1;">
                <h1 id="title">FAIR Report</h1>
            </div>
        </div>
        <div id='metadata_and_canvas'>

            <div>
                <canvas id="tree_diagram" width="400" height="100"></canvas> 
            </div>
            <div id="metadata_table">
                {metadata}
            </div>
            
        </div>
    </div>
</body>

'''

with open('C:/Users/theon/Desktop/report.html', 'w+') as f:
    f.write(HTML_REPORT)

In [ ]:
'''
FairModel
    Just one
FairAddititveMetaModel
    Add all the risks together and display with components
FairComparativeMetaModel
    Put risks side by side
    
Mu, Sigma for Risk

Model Params

Source
'''

'''

Database serialization
Benefits random seed
Loader
Reporter

Report Table JSON
Model Table JSON

'''

In [ ]:
metadata = pd.Series(
    {
        'Author': os.environ['USERNAME'],
        'Generated': str(pd.datetime.now()).partition('.')[0],
        'PyFair Version': pyfair.VERSION
    }
).to_frame()

metadata

In [ ]:
#### Left third, provided data. RIght 2/3rds, tree
model2 = pyfair.read_json(results)
params = model2.export_params()
data = model2.export_results()
del params['Creation Datetime']
df = pd.DataFrame.from_dict(params)
df = df.reindex(['low', 'mode', 'high', 'p'])
df.index = df.index.str.title()
df

In [ ]:
derived = df.agg([np.mean, np.std, np.min, np.max])
derived.index = ['μ', 'σ', 'Minimum', 'Maximum']

final_data = pd.concat([derived, df], axis=0)
final_data = final_data.dropna()

# If Column as data greater than 1,
condition = final_data.max() > 1

# Get True Clumns
true_col = condition.loc[condition].index.values

#Get False COlumns
false_col = condition.loc[~condition].index.values

# If false, it's between 0 and 1. 3 decimals
final_data[false_col] = final_data[false_col]
final_data[false_col] = final_data[false_col].applymap(lambda x: '{0:.3f}'.format(x))

# If true, get the max digits and round
final_data[true_col] = final_data[true_col].astype(int)
final_data[true_col] = final_data[true_col].applymap(lambda x: '{0:,}'.format(x))

for column in ['Probable Loss Magnitude', 'Risk']:
    if column in final_data.columns:
        final_data[column] = '$' + final_data[column]

final_data.T

In [ ]:
model3 = pyfair.read_json(results)
df3 = model3.export_results()
risk = df3['Risk']
risk_d = risk.agg([np.mean, np.std, np.min, np.max]).astype(int)
risk_d.index = ['μ', 'σ', 'Minimum', 'Maximum']

pd.DataFrame(risk_d).T.applymap(lambda x: '${0:,}'.format(x))

In [ ]:
# Draw curves?
model3.__class__

In [ ]:
dollar_format_string     = '${0:,}'
integer_format_string    = '{0:,}'
percentage_format_string = '{0:.3f}'
format_strings = {
    'Risk'                        : dollar_format_string,
    'Loss Event Frequency'        : integer_format_string,
    'Threat Event Frequency'      : integer_format_string,
    'Vulnerability'               : integer_format_string,         
    'Contact'                     : percentage_format_string,
    'Action'                      : percentage_format_string,
    'Threat Capability'           : percentage_format_string,
    'Control Strength'            : percentage_format_string,
    'Probable Loss Magnitude'     : dollar_format_string,
    'Primary Loss Factors'        : dollar_format_string,
    'Asset Loss Factors'          : dollar_format_string,
    'Threat Loss Factors'         : dollar_format_string,
    'Secondary Loss Factors'      : dollar_format_string,
    'Organizational Loss Factors' : dollar_format_string,
    'External Loss Factors'       : dollar_format_string,
}

SINGLE
    header
    meta
    left third, risk top then params. right 2/3, tree diagram
    left have loss excedence, right half exceedence probability
    code
    
COMPARISON
    header
    meta
    left third, risk top then params for each. right 2/3, tree diagram
    left have loss excedence, right half exceedence probability
    code

ADDITIVE
    header
    meta
    for each:
        left third, risk top then params. right 2/3, tree diagram
        left have loss excedence, right half exceedence probability
    for all:
        loss exceedence and exceedence probability and 
        risk
    code

In [ ]:
class FairBasic()

In [ ]:
uuid.uuid1()

In [ ]:
import os
import subprocess
import tempfile

td = tempfile.mkdtemp()
tf = tempfile.NamedTemporaryFile(dir=td, mode='w+')
os.chdir(td)

tf.write('LATEX!')

os.listdir(td)

p = subprocess.run(['dir', str(td)])

In [ ]:
p.read()

In [ ]:
td